<a href="https://colab.research.google.com/github/veritaem/Project2/blob/master/Kaggle_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#first up is some imports

In [1]:
!pip install category_encoders

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, PolynomialFeatures
from sklearn.metrics import precision_score, r2_score, mean_squared_error, classification_report, confusion_matrix, mean_absolute_error, recall_score, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
import seaborn as sns
import category_encoders as ce

In [0]:
pd.set_option('display.height', 3000)
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns', 3000)
pd.set_option('display.width', 3000)

In [0]:
df1 = pd.read_csv('https://raw.githubusercontent.com/veritaem/Project2/master/train_features.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/veritaem/Project2/master/train_labels.csv')
testdf = pd.read_csv('https://raw.githubusercontent.com/veritaem/Project2/master/test_features.csv')

#next up is to look at the data

In [5]:
df1.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [6]:
df2.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [7]:
df = df1.join(df2, rsuffix = 'status_group')
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,idstatus_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,69572,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,8776,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,34310,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,67743,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,19728,functional


In [0]:
id = pd.DataFrame(testdf['id'])

In [9]:
df.shape

(59400, 42)

In [10]:
df['status_group'].unique()

array(['functional', 'non functional', 'functional needs repair'],
      dtype=object)

In [11]:
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [12]:
testdf.shape

(14358, 40)

In [0]:
nan_values_list = ['Not Known', 'Unknown', 'None', 'Not known', 'not known',
                  '-', 'unknown', 'Unknown Installer', '##', 'none', '0']

df = df.replace(nan_values_list, np.nan)
testdf = testdf.replace(nan_values_list, np.nan)

df = df.replace(np.nan, 'unknown')
testdf = testdf.replace(np.nan, 'unknown')

In [0]:
#df['subvillage'] = df['subvillage'].replace(np.nan, 'unknown')
#df['funder'] = df['funder'].replace(np.nan, 'unknown')
#df['installer'] = df['installer'].replace(np.nan, 'unknown')
#df['scheme_management'] = df['scheme_management'].replace(np.nan, 'unknown')
#df['permit'] = df['permit'].replace(np.nan, 'unknown')

#testdf['subvillage'] = testdf['subvillage'].replace(np.nan, 'unknown')
#testdf['funder'] = testdf['funder'].replace(np.nan, 'unknown')
#testdf['installer'] = testdf['installer'].replace(np.nan, 'unknown')
#testdf['scheme_management'] = testdf['scheme_management'].replace(np.nan, 'unknown')
#testdf['permit'] = testdf['permit'].replace(np.nan, 'unknown')

In [0]:
#df['permit'] = df['permit'].replace({'True': 2, 'False': 0, 'unknown': 1})

In [16]:
df.isna().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
s

In [17]:
yencode = df['status_group'].replace({'functional':2, 'functional needs repair':1, 'non functional':0})
yencode = pd.DataFrame(yencode)
yencode.head()

,status_group
0,2
1,2
2,2
3,0
4,2


In [0]:
'''because certain columns seem unimportant, redundant, 
or missing most of their values, 
I will drop them in this cell, to be modified as needed later'''
df2 = df.drop(['scheme_name', 'public_meeting', 'payment_type', 'region', 'idstatus_group', 'recorded_by', 'wpt_name', 'latitude', 'longitude'],axis = 1)
testdf = testdf.drop(['scheme_name', 'public_meeting', 'payment_type', 'region', 'recorded_by', 'wpt_name', 'latitude', 'longitude'],axis = 1)

# ok, time for baselines


In [19]:
yencode.status_group.mean()

1.1588383838383838

In [20]:
baseline = [yencode.status_group.mean()] * len(yencode.status_group)  
mean_absolute_error(yencode.status_group, baseline)

0.9136374604632181

In [21]:
maj_classification = yencode.status_group.mode()
y_pred = np.full(shape=yencode['status_group'].shape, fill_value=maj_classification)
recall_score(yencode['status_group'], y_pred, average = 'micro')

0.543080808080808

#bins and engineering?

In [22]:
'''I see two values that would really ebnefit from a bin, and those are 
population and construction year, lets do it!'''

'I see two values that would really ebnefit from a bin, and those are \npopulation and construction year, lets do it!'

In [0]:
year_bins = [-1, 1980, 1990, 2000, 2010, 2020]
year_labels = [1, 2, 3, 4, 5]
df2['year_made'] = pd.cut(df2['construction_year'], bins = year_bins, labels = year_labels)
testdf['year_made'] = pd.cut(testdf['construction_year'], bins = year_bins, labels = year_labels)

In [0]:
pop_bins = [-1, 10, 20, 100, 250, 1000, 5000, 10000, 100000]
pop_labels = [1, 2, 3, 4, 5, 6, 7, 8]
df2['pop'] = pd.cut(df2['population'], bins = pop_bins, labels = pop_labels)
testdf['pop'] = pd.cut(testdf['population'], bins = pop_bins, labels = pop_labels)

Alright, lets see what we can see! *woosh woosh woosh*

In [25]:
df2.describe(include = 'all')

,id,amount_tsh,date_recorded,funder,gps_height,installer,num_private,basin,subvillage,region_code,district_code,lga,ward,population,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,year_made,pop
count,59400.000000,59400.000000,59400,59400,59400.000000,59400,59400.000000,59400,59400,59400.000000,59400.000000,59400,59400,59400.000000,59400,59400,59400.000000,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400.0,59400.0
unique,NaN,NaN,356,1894,NaN,2139,NaN,9,19287,NaN,NaN,125,2092,NaN,12,3,NaN,18,13,7,12,5,7,8,6,5,5,10,7,3,7,6,3,5.0,8.0
top,NaN,NaN,2011-03-15,Government Of Tanzania,NaN,DWE,NaN,Lake Victoria,Madukani,NaN,NaN,Njombe,Igosi,NaN,VWC,True,NaN,gravity,gravity,gravity,vwc,user-group,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,1.0,1.0
freq,NaN,NaN,572,9084,NaN,17402,NaN,10248,508,NaN,NaN,2503,307,NaN,36793,38852,NaN,26780,26780,26780,40507,52490,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625,32259,26464.0,28690.0
mean,37115.131768,317.650385,NaN,NaN,668.297239,NaN,0.474141,NaN,NaN,15.297003,5.629747,NaN,NaN,179.909983,NaN,NaN,1300.652475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,21453.128371,2997.574558,NaN,NaN,693.116350,NaN,12.236230,NaN,NaN,17.587406,9.633649,NaN,NaN,471.482176,NaN,NaN,951.620547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,NaN,NaN,-90.000000,NaN,0.000000,NaN,NaN,1.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,18519.750000,0.000000,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,5.000000,2.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,37061.500000,0.000000,NaN,NaN,369.000000,NaN,0.000000,NaN,NaN,12.000000,3.000000,NaN,NaN,25.000000,NaN,NaN,1986.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,55656.500000,20.000000,NaN,NaN,1319.250000,NaN,0.000000,NaN,NaN,17.000000,5.000000,NaN,NaN,215.000000,NaN,NaN,2004.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df2['installer'].value_counts()
# i wanted to reduce the number of uniwue variables in each column, and it looks like a jackpot!
#a bunch of these look similar to each other (i.e. gover, government of Tanzania, Gover Dept water, etc)


DWE                               17402
unknown                            4481
Government                         1825
RWE                                1206
Commu                              1060
DANIDA                             1050
KKKT                                898
Hesawa                              840
TCRS                                707
Central government                  622
CES                                 610
Community                           553
DANID                               552
District Council                    551
HESAWA                              539
LGA                                 408
World vision                        408
WEDECO                              397
TASAF                               396
District council                    392
Gover                               383
AMREF                               329
TWESA                               316
WU                                  301
Dmdd                                287


In [0]:
red_list = ['Government', 'Central government', 'Central Govt', 'GOVER', 'Gover',
            'Gove', 'Centr', 'Central Government', 'Wizara ya maji', 'Water', 
            'GOVERNMENT', 'Tanzania Government', 'GOVERN', 'central government',
           'ISF/Government', 'CENTRAL GOVERNMENT', 'MINISTRY OF WATER', 
            'Ministry of water engineer', 'Cental Government', 'ADRA/Government']

list2 = ['World vision', 'World Vision', 'WORLD VISION', 'World Vission', 
        'World Division', 'Government /World Vision', 'World visiin']


df2['installer'] = df2['installer'].replace(red_list, 'government')
df2['installer'] = df2['installer'].replace(list2, 'World Vision')

testdf['installer'] = testdf['installer'].replace(red_list, 'government')
testdf['installer'] = testdf['installer'].replace(list2, 'World Vision')

In [28]:
df2.groupby(['water_quality', 'basin']).count()

id  amount_tsh  date_recorded  funder  gps_height  installer  num_private  subvillage  region_code  district_code   lga  ward  population  scheme_management  permit  construction_year  extraction_type  extraction_type_group  extraction_type_class  management  management_group  payment  quality_group  quantity  quantity_group  source  source_type  source_class  waterpoint_type  waterpoint_type_group  status_group  year_made   pop
water_quality      basin                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
coloured           Internal                   66          66             66      66          66         66           66          66           66             66    66    66          66                 66      66                 66               66                     66                     66          66                66       66             66        66              66      66           66            66               66                     66            66         66    66
                   Lake Nyasa                  7           7              7       7           7          7            7           7            7              7     7     7           7                  7       7                  7                7                      7                      7           7                 7        7              7         7               7       7            7             7                7                      7             7          7     7
                   Lake Rukwa                  8           8              8       8           8          8            8           8            8              8     8     8           8                  8       8                  8                8                      8                      8           8                 8        8              8         8               8       8            8             8                8                      8             8          8     8
                   Lake Tanganyika            64          64             64      64          64         64           64          64           64             64    64    64          64                 64      64                 64               64                     64                     64          64                64       64             64        64              64      64           64            64               64                     64            64         64    64
                   Lake Victoria              49          49             49      49          49         49           49          49           49             49    49    49          49                 49      49                 49               49                     49                     49          49                49       49             49        49              49      49           49            49               49                     49            49         49    49
                   Pangani                    35          35             35      35          35         35           35          35           35             35    35    35          35                 35      35                 35               35                     35                     35          35                35       35             35        35              35      35           35            35               35                     35            35         35    35
                   Rufiji                    183         183            183     183         183        183          183         183          183            183   183   183         183                183     183    

In [29]:
df2.groupby(['status_group', 'water_quality']).count()

id  amount_tsh  date_recorded  funder  gps_height  installer  num_private  basin  subvillage  region_code  district_code    lga   ward  population  scheme_management  permit  construction_year  extraction_type  extraction_type_group  extraction_type_class  management  management_group  payment  quality_group  quantity  quantity_group  source  source_type  source_class  waterpoint_type  waterpoint_type_group  year_made    pop
status_group            water_quality                                                                                                                                                                                                                                                                                                                                                                                                                                                      
functional              coloured              246         246            246     246         246        246          246    246         246          246            246    246    246         246                246     246                246              246                    246                    246         246               246      246            246       246             246     246          246           246              246                    246        246    246
                        fluoride              151         151            151     151         151        151          151    151         151          151            151    151    151         151                151     151                151              151                    151                    151         151               151      151            151       151             151     151          151           151              151                    151        151    151
                        fluoride abandoned      6           6              6       6           6          6            6      6           6            6              6      6      6           6                  6       6                  6                6                      6                      6           6                 6        6              6         6               6       6            6             6                6                      6          6      6
                        milky                 438         438            438     438         438        438          438    438         438          438            438    438    438         438                438     438                438              438                    438                    438         438               438      438            438       438             438     438          438           438              438                    438        438    438
                        salty                2220        2220           2220    2220        2220       2220         2220   2220        2220         2220           2220   2220   2220        2220               2220    2220               2220             2220                   2220                   2220        2220              2220     2220           2220      2220            2220    2220         2220          2220             2220                   2220       2220   2220
                        salty abandoned       174         174            174     174         174        174          174    174         174          174            174    174    174         174                174     174                174              174                    174                    174         174               174      174            174       174             174     174          174           174              174                    174        174    174
                        soft                28760       28760          28760   28760       28760      28760        28760  28760       28760        28760          28760  28760  28760       28760              28760   28760              28760

In [30]:
df2.groupby(['management', 'status_group']).count()

id  amount_tsh  date_recorded  funder  gps_height  installer  num_private  basin  subvillage  region_code  district_code    lga   ward  population  scheme_management  permit  construction_year  extraction_type  extraction_type_group  extraction_type_class  management_group  payment  water_quality  quality_group  quantity  quantity_group  source  source_type  source_class  waterpoint_type  waterpoint_type_group  year_made    pop
management       status_group                                                                                                                                                                                                                                                                                                                                                                                                                                                               
company          functional                 267         267            267     267         267        267          267    267         267          267            267    267    267         267                267     267                267              267                    267                    267               267      267            267            267       267             267     267          267           267              267                    267        267    267
                 functional needs repair     15          15             15      15          15         15           15     15          15           15             15     15     15          15                 15      15                 15               15                     15                     15                15       15             15             15        15              15      15           15            15               15                     15         15     15
                 non functional             403         403            403     403         403        403          403    403         403          403            403    403    403         403                403     403                403              403                    403                    403               403      403            403            403       403             403     403          403           403              403                    403        403    403
other            functional                 505         505            505     505         505        505          505    505         505          505            505    505    505         505                505     505                505              505                    505                    505               505      505            505            505       505             505     505          505           505              505                    505        505    505
                 functional needs repair     55          55             55      55          55         55           55     55          55           55             55     55     55          55                 55      55                 55               55                     55                     55                55       55             55             55        55              55      55           55            55               55                     55         55     55
                 non functional             284         284            284     284         284        284          284    284         284          284            284    284    284         284                284     284                284              284                    284                    284               284      284            284            284       284             284     284          284           284              284                    284        284    284
other - school   functional                  23          23             23      23          23         23           23     23          23           23             23     23     23          23                 23      23           

In [31]:
df2.groupby(['water_quality', 'status_group']).count()


id  amount_tsh  date_recorded  funder  gps_height  installer  num_private  basin  subvillage  region_code  district_code    lga   ward  population  scheme_management  permit  construction_year  extraction_type  extraction_type_group  extraction_type_class  management  management_group  payment  quality_group  quantity  quantity_group  source  source_type  source_class  waterpoint_type  waterpoint_type_group  year_made    pop
water_quality      status_group                                                                                                                                                                                                                                                                                                                                                                                                                                                            
coloured           functional                 246         246            246     246         246        246          246    246         246          246            246    246    246         246                246     246                246              246                    246                    246         246               246      246            246       246             246     246          246           246              246                    246        246    246
                   functional needs repair     54          54             54      54          54         54           54     54          54           54             54     54     54          54                 54      54                 54               54                     54                     54          54                54       54             54        54              54      54           54            54               54                     54         54     54
                   non functional             190         190            190     190         190        190          190    190         190          190            190    190    190         190                190     190                190              190                    190                    190         190               190      190            190       190             190     190          190           190              190                    190        190    190
fluoride           functional                 151         151            151     151         151        151          151    151         151          151            151    151    151         151                151     151                151              151                    151                    151         151               151      151            151       151             151     151          151           151              151                    151        151    151
                   functional needs repair     13          13             13      13          13         13           13     13          13           13             13     13     13          13                 13      13                 13               13                     13                     13          13                13       13             13        13              13      13           13            13               13                     13         13     13
                   non functional              36          36             36      36          36         36           36     36          36           36             36     36     36          36                 36      36                 36               36                     36                     36          36                36       36             36        36              36      36           36            36               36                     36         36     36
fluoride abandoned functional                   6           6              6       6           6          6            6      6           6            6              6      6      6           6                  6       6                  6

In [32]:
df2['status_group'].groupby(df2['water_quality']).count()

water_quality
coloured                490
fluoride                200
fluoride abandoned       17
milky                   804
salty                  4856
salty abandoned         339
soft                  50818
unknown                1876
Name: status_group, dtype: int64

#X and y, pipe it!

In [0]:
X = df2.drop(['status_group', 'funder', 'installer', 'subvillage', 'population', 'construction_year', 'ward', 'num_private',
             'region_code', 'extraction_type_group', 'quantity_group', 'source_class', 'source_type'], axis = 1)
y = df2['status_group']

Xtdf = testdf.drop(['funder', 'installer', 'subvillage', 'population', 'construction_year', 'ward', 'num_private',
                   'region_code', 'extraction_type_group', 'quantity_group', 'source_class', 'source_type'], axis = 1)

In [34]:
X.describe(include = 'all')

,id,amount_tsh,date_recorded,gps_height,basin,district_code,lga,scheme_management,permit,extraction_type,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,source,waterpoint_type,waterpoint_type_group,year_made,pop
count,59400.000000,59400.000000,59400,59400.000000,59400,59400.000000,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400.0,59400.0
unique,NaN,NaN,356,NaN,9,NaN,125,12,3,18,7,12,5,7,8,6,5,10,7,6,5.0,8.0
top,NaN,NaN,2011-03-15,NaN,Lake Victoria,NaN,Njombe,VWC,True,gravity,gravity,vwc,user-group,never pay,soft,good,enough,spring,communal standpipe,communal standpipe,1.0,1.0
freq,NaN,NaN,572,NaN,10248,NaN,2503,36793,38852,26780,26780,40507,52490,25348,50818,50818,33186,17021,28522,34625,26464.0,28690.0
mean,37115.131768,317.650385,NaN,668.297239,NaN,5.629747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,21453.128371,2997.574558,NaN,693.116350,NaN,9.633649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,NaN,-90.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,18519.750000,0.000000,NaN,0.000000,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,37061.500000,0.000000,NaN,369.000000,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,55656.500000,20.000000,NaN,1319.250000,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
testdf.describe(include = 'all')

,id,amount_tsh,date_recorded,funder,gps_height,installer,num_private,basin,subvillage,region_code,district_code,lga,ward,population,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,year_made,pop
count,14358.000000,14358.000000,14358,14358,14358.0000,14358,14358.000000,14358,14358,14358.000000,14358.000000,14358,14358,14358.000000,14358,14358,14358.000000,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358.0,14358.0
unique,NaN,NaN,331,957,NaN,1053,NaN,9,8254,NaN,NaN,124,1934,NaN,12,3,NaN,17,13,7,12,5,7,8,6,5,5,10,7,3,7,6,5.0,8.0
top,NaN,NaN,2011-03-16,Government Of Tanzania,NaN,DWE,NaN,Lake Victoria,Shuleni,NaN,NaN,Njombe,Igosi,NaN,VWC,True,NaN,gravity,gravity,gravity,vwc,user-group,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,communal standpipe,communal standpipe,1.0,1.0
freq,NaN,NaN,137,2117,NaN,4162,NaN,2535,136,NaN,NaN,611,79,NaN,8807,9442,NaN,6168,6168,6168,9780,12639,6098,12237,12237,7997,7997,4211,4211,11127,6790,8260,6353.0,6972.0
mean,37232.859799,324.219996,NaN,NaN,653.6363,NaN,0.408971,NaN,NaN,15.156359,5.713052,NaN,NaN,187.055439,NaN,NaN,1298.251985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,21382.890432,2533.367778,NaN,NaN,688.2721,NaN,8.231859,NaN,NaN,17.387588,9.794304,NaN,NaN,476.065978,NaN,NaN,952.551852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,10.000000,0.000000,NaN,NaN,-57.0000,NaN,0.000000,NaN,NaN,1.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,18765.500000,0.000000,NaN,NaN,0.0000,NaN,0.000000,NaN,NaN,5.000000,2.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,37442.000000,0.000000,NaN,NaN,346.0000,NaN,0.000000,NaN,NaN,12.000000,3.000000,NaN,NaN,25.000000,NaN,NaN,1986.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,55909.250000,25.000000,NaN,NaN,1306.0000,NaN,0.000000,NaN,NaN,17.000000,5.000000,NaN,NaN,230.000000,NaN,NaN,2004.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [37]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(testdf.shape)
print(Xtdf.shape)

(44550, 22)
(14850, 22)
(44550,)
(14850,)
(14358, 34)
(14358, 22)


In [38]:
print(X_train.dtypes)
print(y_train.dtypes)

id                          int64
amount_tsh                float64
date_recorded              object
gps_height                  int64
basin                      object
district_code               int64
lga                        object
scheme_management          object
permit                     object
extraction_type            object
extraction_type_class      object
management                 object
management_group           object
payment                    object
water_quality              object
quality_group              object
quantity                   object
source                     object
waterpoint_type            object
waterpoint_type_group      object
year_made                category
pop                      category
dtype: object
object


In [39]:
Xtdf.shape

(14358, 22)

In [0]:
pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    PolynomialFeatures(2),
    RobustScaler(),
    LogisticRegression()
)

In [0]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

In [0]:
subby = pipeline.predict(Xtdf)
print(subby.shape)
subby = pd.DataFrame(subby)
subs = id.join(subby, rsuffix = '0')
subs = subs.rename(index=str, columns={0:'status_group'})
print(subs.shape)
subs.head()

In [0]:
subs.to_csv('sub1.csv', index=False)

In [0]:
#param_grid = {
 #   'logisticregression__C': [.01, .1, 1, 10, 100, 1000],
  #  'logisticregression__class_weight':['balanced', None],
   # 'selectkbest__k': range(1, 6)}


#gs = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
    #              scoring='recall', 
     #             verbose=10)
#gs.fit(X_train, y_train)

In [0]:
#validation_score = gs.best_score_

#print('Cross-Validation Score:', validation_score)

#print('Best estimator:', gs.best_estimator_)

In [0]:
#selector = gs.best_estimator_.named_steps['polynomialfeatures']
#all_names = X_train.columns
#selected_mask = selector.get_feature_names()


#print('Features selected:')
#for name in selected_mask:
#    print(name)

In [0]:
#y_pred = gs.predict(X_test)
#y_pred

In [0]:
#test_score = mean_absolute_error(y_test, y_pred)
#print('Test Score:', test_score)